<a href="https://colab.research.google.com/github/cubecloud/dogs_vs_cats/blob/master/dogs_vs_cats_v_1_0_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%tensorflow_version 2.x
%tensorflow_version 1.15.2
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
#%tensorflow_version 2.x
%tensorflow_version 1.15.2
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Found GPU at: /device:GPU:0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#%tensorflow_version 2.x
%tensorflow_version 1.15.2
# Load the TensorBoard notebook extension
%load_ext tensorboard
import os
import glob
import pytz
import datetime
from math import exp
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from google.colab import drive
drive.mount('/content/drive')
tf.__version__

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.2`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'1.15.2'

In [0]:
version=109
timezone = pytz.timezone("Europe/Moscow")
image_size = 150
batch_size = 40
epochs = 150
start_learning_rate = 0.001
start_patience = round(epochs*0.04)

print (f'Image Size = {image_size}x{image_size}')

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    # x = data_augmentation(inputs)
    x = inputs
    # Entry block
    # x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization( momentum=0.9 )(x) # added momentum [ 0.9 ]
    x = layers.Activation("relu")(x)

    x = layers.Dropout(0.5)(x) #<= added dropout layer [ ]

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization( momentum=0.9 )(x) # added momentum [ 0.9 ]
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)

        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization( momentum=0.9)(x) # added momentum [ 0.9 ]
        x = layers.Activation("relu")(x)
        
        # x = layers.Dropout(0.5)(x) #<= added dropout layer [ ]
        
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization( momentum=0.9 )(x) # added momentum [ 0.9 ]
        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization( momentum=0.9 )(x) # added momentum [ 0.9 ]
    x = layers.Activation("relu")(x)

    x = layers.Dropout(0.5)(x) #<= added dropout layer [ 0.2 0.5 ]
    
    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.6)(x) #<= dropout [ 0.5, 0.52, 0.5, 0.6 ,0.6 ]
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

home_dir = "/content/drive/My Drive/Colab Notebooks/dogs_vs_cats/"
base_dir = "/content/drive/My Drive/Colab Notebooks/dogs_vs_cats/data/"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# data loading
# ----------------------------------------------------

x_train = np.load(os.path.join(home_dir, 'dogs_vs_cats_photos_train.npy'))
y_train = np.load(os.path.join(home_dir,'dogs_vs_cats_labels_train.npy'))[2:,]
x_validation = np.load(os.path.join(home_dir,'dogs_vs_cats_photos_validation.npy'))
y_validation = np.load(os.path.join(home_dir,'dogs_vs_cats_labels_validation.npy'))[2:,]
print (x_train.shape, y_train.shape)
print (x_validation.shape, y_validation.shape)
# ----------------------------------------------------

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30, # <= [ 40, 30 ]
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode="nearest")

validation_datagen = ImageDataGenerator(rescale=1./255)

train_datagen.fit(x_train)

train_generator = train_datagen.flow(
    x=x_train,
    y=y_train,
    shuffle=True,
    batch_size=batch_size)

validation_generator = validation_datagen.flow(
    x=x_validation,
    y=y_validation,
    shuffle=True,
    batch_size=batch_size)

model = make_model(input_shape=(image_size,image_size) + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)
model.summary()

def scheduler(epoch):
  if epoch < 10:
    return start_learning_rate
  else:
    # return start_learning_rate * exp(0.1 * (10 - epoch))
    return start_learning_rate*(0.75**(epoch//10))

class EarlyStoppingAtMinValLoss(tf.keras.callbacks.Callback):
  def __init__(self, start_learning_rate=start_learning_rate, patience=3):
    super(EarlyStoppingAtMinValLoss, self).__init__()
    self.patience = patience
    self.start_learning_rate = start_learning_rate
    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf
  
  def on_epoch_begin(self, epoch, logs=None):
    if (epoch >= 10) and (epoch%10 == 0):
      self.patience += round((epochs-epoch)*0.03)
    # self.patience = round((self.start_learning_rate / scheduler (epoch) + (epochs*0.15))/2)
    print (f'Patience: {self.patience}')

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('val_loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

  # def on_train_batch_begin(self, batch, logs=None):
  #   print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now(timezone).time()))


lrs = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
log_dir = os.path.join(home_dir, 'logs/fit')
esmvl = EarlyStoppingAtMinValLoss (patience=start_patience)
es = EarlyStopping(monitor='val_loss', mode='min', patience=start_patience, verbose=1)
tb = TensorBoard(log_dir=log_dir, histogram_freq=10, write_graph=True, write_images=False )

def filename_with_time (prefix_str, root_str):
  return prefix_str + root_str + str(datetime.datetime.now(timezone).strftime("%d-%m-%Y_%H-%M"))

model_name= f'cats_and_dogs_{str(version)}_{str(image_size)}x{str(image_size)}_{str(batch_size)}_'
chkp = keras.callbacks.ModelCheckpoint(os.path.join(home_dir,'save/', filename_with_time(model_name,"save_at_{epoch:02d}-{val_loss:.4f}_") + ".h5"), monitor='val_loss', save_best_only=True)

model.compile(
    optimizer=keras.optimizers.Adam(lr=start_learning_rate),
    loss="binary_crossentropy",
    metrics=["accuracy"],
    )

callbacks = [lrs,  esmvl, chkp ]

start = datetime.datetime.now()
history = model.fit(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = validation_generator,
    validation_steps = len(validation_generator),
    epochs = epochs,
    verbose=1,
    callbacks = callbacks)

end = datetime.datetime.now()
print (f'Planned epochs: {epochs} Calculated epochs : {len(history.history["loss"])} Time elapsed: {end - start}')
epochs = len(history.history['loss'])
model_name= f'cats_and_dogs_{str(version)}_{str(image_size)}x{str(image_size)}_{str(batch_size)}_epochs-{epochs}_'
model.save(os.path.join(home_dir, model_name + str(datetime.datetime.now(timezone).strftime("%d-%m-%Y_%H-%M")) + '.h5'))

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15,12))
gs = fig.add_gridspec(1, 1)
ax1 = fig.add_subplot()
# Don't allow the axis to be on top of your data
ax1.set_axisbelow(True)
# Turn on the minor TICKS, which are required for the minor GRID
ax1.minorticks_on()
# Customize the major grid
ax1.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
# Customize the minor grid
ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')
N = np.arange(0, len(history.history["acc"]) )
plt.plot(N, history.history["loss"], linestyle='-', color='red', label="Training loss")
plt.plot(N, history.history["acc"], linestyle='-.', color='magenta', label="Training accuracy")
plt.plot(N, history.history["val_loss"], linestyle='--', color='blue', label="Validation loss")
plt.plot(N, history.history["val_acc"], linestyle=':', color='violet', label="Validation accuracy")
plt.title("Training/validation Loss and Accuracy")
plt.legend()
plt.savefig(os.path.join(home_dir,'figures', ('fig_' + model_name + str(datetime.datetime.now(timezone).strftime("%d-%m-%Y_%H-%M"))+'.png')))
plt.close()

Image Size = 150x150
(18800, 150, 150, 3) (18800,)
(6200, 150, 150, 3) (6200,)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 75, 75, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 75, 75, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (

In [0]:
test_filenames = os.listdir("../input/test1/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "../input/test1/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })


sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("../input/test1/test1/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()


In [0]:
%tensorboard --logdir='log_dir'

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/fit 

In [0]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15,12))
gs = fig.add_gridspec(1, 1)
ax1 = fig.add_subplot()
# Don't allow the axis to be on top of your data
ax1.set_axisbelow(True)
# Turn on the minor TICKS, which are required for the minor GRID
ax1.minorticks_on()
# Customize the major grid
ax1.grid(which='major', linestyle='-', linewidth='0.5', color='gray')
# Customize the minor grid
ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')
N = np.arange(0, len(history.history["acc"]) )
plt.plot(N, history.history["loss"], linestyle='-', color='red', label="Training loss")
plt.plot(N, history.history["acc"], linestyle='-.', color='magenta', label="Training accuracy")
plt.plot(N, history.history["val_loss"], linestyle='--', color='blue', label="Validation loss")
plt.plot(N, history.history["val_acc"], linestyle=':', color='violet', label="Validation accuracy")
plt.title("Training/validation Loss and Accuracy")
plt.legend()
plt.show()
plt.savefig(os.path.join(home_dir,'figures', ('fig_' + model_name + str(datetime.datetime.now(timezone).strftime("%d-%m-%Y_%H-%M"))+'.png')))
plt.close()

In [0]:
tensorboard --inspect --logdir='log_dir'

SyntaxError: ignored